<center><img src='https://thecitymagazineelp.com/wp-content/uploads/2019/03/AdobeStock_103569152-5BConverted5D-1000.jpg' height=350></center>
<p>
<h1><center> Normal VS Cataract VS Glaucoma </center></h1>

## 1.Objectifs

* L’objectif ici est de développer un réseau CNN en se basant sur des architecture différentes (vgg16,vgg19,EfficientNetB7,DenseNet201) pour la classification de deux maladie (Normal VS Cataract VS Glaucoma)

## 2. Préparation de la base de données

### 2.1 importer les bibliothèques nécessaires

In [ ]:
import numpy as np
import pandas as pd
import os
import random, re, math
import tensorflow as tf, tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as L
from tensorflow.keras.applications import ResNet152V2, InceptionResNetV2, InceptionV3, Xception, VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from keras import regularizers

import matplotlib.pyplot as plt

!pip install efficientnet
import efficientnet.tfkeras as efn

## 2.2 Configuration de tpu

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)


In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('ocular-disease-recognition-odir5k')

In [ ]:
train = pd.read_csv('../input/new-df-csv-oc/new_df_oc.csv')
train_paths = train.filename.apply(lambda x: GCS_DS_PATH+ '/ODIR-5K/ODIR-5K/Training Images/' + x).values
train_labels = train.target.values

In [ ]:
train.head(10)

In [ ]:
train=train.drop(columns=['D','M','H','A','O'],axis=1)

### 2.3 Normal Vs Cataract

In [ ]:
train=train[((train['N']== 1) | (train['C'] == 1)| (train['G'] == 1))]

In [ ]:
train

## 2.4 Diviser notre dataset en 80% l'entraînement et 20% pour le test

In [ ]:
train,valid = train_test_split(train,test_size = 0.2,random_state = 42)

## 3 Hyperparamètre

In [ ]:
BATCH_SIZE = 8* strategy.num_replicas_in_sync
img_size = 512
EPOCHS = 20
SEED = 42

## 4. Prétraitement des données

In [ ]:
def decode_image(filename, label=None, image_size=(img_size,img_size)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3) 
    image = tf.image.resize(image, image_size)
    image = tf.cast(image, tf.float32)
    image = tf.image.per_image_standardization(image)
    if label is None:
        return image
    else:
        return image, label
    
def preprocess(df,test=False):
    paths = df.filename.apply(lambda x: GCS_DS_PATH + '/ODIR-5K/ODIR-5K/Training Images/' + x).values
    labels = df.loc[:, ['N', 'C', 'G']].values
    if test==False:
        return paths,labels
    else:
        return paths
    
def data_augment(image, label=None, seed=SEED):
    image = tf.image.random_flip_left_right(image, seed=seed)
    image = tf.image.random_flip_up_down(image, seed=seed)
           
    if label is None:
        return image
    else:
        return image, label

## 5. Augmentation

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.

    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )

    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

In [ ]:
def transform(image,label=None):
    DIM = img_size
    XDIM = DIM%2 
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 8. * tf.random.normal([1],dtype='float32') 
    w_shift = 8. * tf.random.normal([1],dtype='float32') 
  
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
              
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
    
    if label is None:
        return tf.reshape(d,[DIM,DIM,3])
    else:
        return tf.reshape(d,[DIM,DIM,3]),label

## 6.Création d'un générateur pour l'ensemble de données d'entraînement 

In [ ]:
train_dataset = (tf.data.Dataset
    .from_tensor_slices(preprocess(train))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .map(transform,num_parallel_calls=AUTO)
    .shuffle(SEED)
    .batch(BATCH_SIZE)
    .repeat()
    .prefetch(AUTO))

## 7. Création d'un générateur pour l'ensemble de données de test

In [ ]:
test_dataset= (tf.data.Dataset
    .from_tensor_slices(preprocess(valid))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO))

## 8. Fonction du taux d'apprentissage

In [ ]:
LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

## 9. Fonction de perte

In [ ]:
def categorical_focal_loss(gamma=2., alpha=.25):
    def categorical_focal_loss_fixed(y_true, y_pred):
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=1)
    return categorical_focal_loss_fixed

<p>
<h1><center> EfficientNetB7 </center></h1>
<center><img src='https://1.bp.blogspot.com/-DjZT_TLYZok/XO3BYqpxCJI/AAAAAAAAEKM/BvV53klXaTUuQHCkOXZZGywRMdU9v9T_wCLcBGAs/s1600/image2.png' height=350></center>
<p>

In [ ]:
with strategy.scope():
    enet = efn.EfficientNetB7(input_shape=(img_size, img_size, 3),weights='noisy-student',include_top=False)

In [ ]:
with strategy.scope():
    for layer in  enet.layers:
        layer.trainable = False

    for i in range(-3,0):
         enet.layers[i].trainable = True

In [ ]:
with strategy.scope():
    ef7 =tf.keras.Sequential()
    ef7.add(enet)
    ef7.add(tf.keras.layers.MaxPooling2D())
    ef7.add(tf.keras.layers.Conv2D(4096,3,padding='same'))
    ef7.add(tf.keras.layers.BatchNormalization())
    ef7.add(tf.keras.layers.ReLU())
    ef7.add(tf.keras.layers.GlobalAveragePooling2D())
    ef7.add(tf.keras.layers.Dropout(0.35))
    ef7.add(tf.keras.layers.Flatten())

    ef7.add(tf.keras.layers.Dense(2048,activation='relu'))
    ef7.add(tf.keras.layers.BatchNormalization())
    ef7.add(tf.keras.layers.LeakyReLU())
    ef7.add(tf.keras.layers.Dropout(0.35))

    ef7.add(tf.keras.layers.Dense(1024,activation='relu'))
    ef7.add(tf.keras.layers.BatchNormalization())
    ef7.add(tf.keras.layers.LeakyReLU())
    ef7.add(tf.keras.layers.Dropout(0.25))
    ef7.add(tf.keras.layers.Dense(3,activation='softmax'))
    ef7.compile(
                optimizer=tf.optimizers.Adam(lr=0.0001),
                loss=categorical_focal_loss(gamma=2., alpha=.25),
                metrics=['categorical_accuracy',
                        tf.keras.metrics.Recall(),
                        tf.keras.metrics.Precision(),   
                        tf.keras.metrics.AUC(),
                        tfa.metrics.F1Score(num_classes=3, average="macro")
                       ])


# 10 Entraînement

In [ ]:
h7=ef7.fit(
    train_dataset,
    steps_per_epoch=train_labels.shape[0] // BATCH_SIZE,
    callbacks=[lr_callback],
    epochs=EPOCHS)

## 11. Affichage des courbes (acc,loss)

In [ ]:
import seaborn as sns
sns.set()
fig = plt.figure(0, (12, 4))

ax = plt.subplot(1, 2, 1)
sns.lineplot(h7.epoch,h7.history['categorical_accuracy'], label = 'train')
plt.title('Accuracy')
plt.tight_layout()

ax = plt.subplot(1, 2, 2)
sns.lineplot(h7.epoch,h7.history['loss'], label = 'train')
plt.title('Loss')
plt.tight_layout()
plt.show()

## 11. Test et évaluation

In [ ]:
ef7.evaluate(test_dataset)

In [ ]:
from sklearn.metrics import confusion_matrix
classes=['N','C','G']
Y_pred = ef7.predict(test_dataset)
true_classes = valid.loc[:, ['N','C','G']].values
print('Confusion Matrix')
cm=confusion_matrix(true_classes.argmax(axis=1),Y_pred.argmax(axis=1))
cm

## 12. Matrice de confusion

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")
import itertools
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Oranges):
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm,classes)

## 13. specificity et sensitivity

In [ ]:
def calculate_sensitivity_specificity(y_test, y_pred_test):
    actual_pos = y_test == 1
    actual_neg = y_test == 0
    
    true_pos = (y_pred_test == 1) & (actual_pos)
    false_pos = (y_pred_test == 1) & (actual_neg)
    true_neg = (y_pred_test == 0) & (actual_neg)
    false_neg = (y_pred_test == 0) & (actual_pos)
    
    # Calculate sensitivity and specificity
    sensitivity = np.sum(true_pos) / np.sum(actual_pos)
    specificity = np.sum(true_neg) / np.sum(actual_neg)
    
    return sensitivity, specificity

In [ ]:
sensitivity, specificity= calculate_sensitivity_specificity(true_classes.argmax(axis=1),Y_pred.argmax(axis=1))
print ('Sensitivity:', sensitivity)
print ('Specificity:', specificity)

<p>
<h1><center> DenseNet201 </center></h1>
<center><img src='https://oi.readthedocs.io/en/latest/_images/cnn_vs_resnet_vs_densenet.png' height=20></center>
<p>


In [ ]:
with strategy.scope():
    DenseNet201 = tf.keras.applications.DenseNet201(input_shape=(512, 512, 3), weights='imagenet', include_top=False)

In [ ]:
with strategy.scope():
    for layer in  DenseNet201.layers:
        layer.trainable = False

    for i in range(-3,0):
         DenseNet201.layers[i].trainable = True

In [ ]:
with strategy.scope():
    model_D201=tf.keras.Sequential()
    model_D201.add(DenseNet201)
    model_D201.add(tf.keras.layers.MaxPooling2D())
    model_D201.add(tf.keras.layers.Conv2D(4096,3,padding='same'))
    model_D201.add(tf.keras.layers.BatchNormalization())
    model_D201.add(tf.keras.layers.ReLU())
    model_D201.add(tf.keras.layers.GlobalAveragePooling2D())
    model_D201.add(tf.keras.layers.Dropout(0.35))
    model_D201.add(tf.keras.layers.Flatten())

    model_D201.add(tf.keras.layers.Dense(2048,activation='relu'))
    model_D201.add(tf.keras.layers.BatchNormalization())
    model_D201.add(tf.keras.layers.LeakyReLU())
    model_D201.add(tf.keras.layers.Dropout(0.35))
    
    model_D201.add(tf.keras.layers.Dense(1024,activation='relu'))
    model_D201.add(tf.keras.layers.BatchNormalization())
    model_D201.add(tf.keras.layers.LeakyReLU())
    model_D201.add(tf.keras.layers.Dropout(0.25))
    model_D201.add(tf.keras.layers.Dense(3,activation='softmax'))
    model_D201.compile(
                optimizer=tf.optimizers.Adam(lr=0.0001),
                loss=categorical_focal_loss(gamma=2., alpha=.25),
                metrics=['categorical_accuracy',
                        tf.keras.metrics.Recall(),
                        tf.keras.metrics.Precision(),   
                        tf.keras.metrics.AUC(),
                        tfa.metrics.F1Score(num_classes=3, average="macro")
                       ])


# 1. Entraînement

In [ ]:
D_201=model_D201.fit(
    train_dataset, 
    steps_per_epoch=train_labels.shape[0] // BATCH_SIZE,
    callbacks=[lr_callback],
    epochs=EPOCHS)
    #validation_data=test_dataset)

## 2.Affichage des courbes (acc,loss)

In [ ]:
import seaborn as sns
sns.set()
fig = plt.figure(0, (12, 4))

ax = plt.subplot(1, 2, 1)
sns.lineplot(D_201.epoch,D_201.history['categorical_accuracy'], label = 'train')
plt.title('Accuracy')
plt.tight_layout()

ax = plt.subplot(1, 2, 2)
sns.lineplot(D_201.epoch,D_201.history['loss'], label = 'train')
plt.title('Loss')
plt.tight_layout()
plt.show()

## 3. Test et évaluation

In [ ]:
model_D201.evaluate(test_dataset)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
Y_pred = model_D201.predict(test_dataset)
true_classes = valid.loc[:, ['N','C','G']].values
print('Confusion Matrix')
cm=confusion_matrix(true_classes.argmax(axis=1),Y_pred.argmax(axis=1))
cm

In [ ]:
plot_confusion_matrix(cm,classes)

## 4. Matrice de confusion

In [ ]:
sensitivity, specificity= calculate_sensitivity_specificity(true_classes.argmax(axis=1),Y_pred.argmax(axis=1))
print ('Sensitivity:', sensitivity)
print ('Specificity:', specificity)

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D,concatenate,Concatenate,multiply, LocallyConnected2D, Lambda)
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
import keras
from keras.models import Model
from keras.activations import hard_sigmoid
CFG = dict(
    inp_size          = 512,
    read_size         = 512, 
    crop_size         = 512,
    net_size          = 512)


In [ ]:
with strategy.scope():
    in_lay = Input(shape=(CFG['inp_size'], CFG['inp_size'],3))
    base_model = efn.EfficientNetB7(weights='noisy-student',
        input_shape=(CFG['inp_size'], CFG['inp_size'],3),
        include_top=False
                       )
    #base_model.load_weights("../input/efficientnet-keras-weights-b0b5/efficientnet-b5_imagenet_1000_notop.h5")
    pt_depth = base_model.get_output_shape_at(0)[-1]
    pt_features = base_model(in_lay)
    bn_features = BatchNormalization()(pt_features)
    
    # ici nous faisons un mécanisme d'attention pour activer et désactiver les pixels dans le GAP
    # lidee est baser sur cette explication 
    #1-http://akosiorek.github.io/ml/2017/10/14/visual-attention.html
    #2-https://machinelearningmastery.com/how-does-attention-work-in-encoder-decoder-recurrent-neural-networks/
    
    
    attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(Dropout(0.5)(bn_features))
    attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
    attn_layer = Conv2D(8, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
    attn_layer = Conv2D(1, 
                        kernel_size = (1,1), 
                        padding = 'valid', 
                        activation = 'sigmoid')(attn_layer)
    # diffusez sur toutes les chaînes
    # kernel_size  détermine les dimensions du noyau. Les dimensions courantes comprennent 1×1, 3×3, 5×5 et 7×7, qui peuvent être passées en (1, 1), (3, 3), (5, 5) ou (7, 7) tuples.
    # Il s'agit d'un nombre entier ou d'un tuple/liste de 2 nombres entiers, spécifiant la hauteur et la largeur de la fenêtre de convolution 2D.
    #  Ce paramètre doit être un nombre entier impair
    # pour plus de details sur cette partie (mask et use_bias ... ) il ya  une bonne explication sur geekforgeeks
    #https://www.geeksforgeeks.org/keras-conv2d-class/
    
    up_c2_w = np.ones((1, 1, 1, pt_depth))
    up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
                   activation = 'linear', use_bias = False, weights = [up_c2_w])
    up_c2.trainable = False
    attn_layer = up_c2(attn_layer)

    mask_features = multiply([attn_layer, bn_features])
    gap_features = GlobalAveragePooling2D()(mask_features)
    gap_mask = GlobalAveragePooling2D()(attn_layer)
    
    # pour tenir compte des valeurs manquantes du modèle d'attention
    # pour bien comprendre resaclegap il ya un bon exemple ici qui explique tellemnt bien cette partie 
    # https://codefellows.github.io/sea-python-401d5/lectures/rescaling_data.html
    
    gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
    gap_dr = Dropout(0.25)(gap)
    dr_steps = Dropout(0.25)(Dense(128, activation = 'relu')(gap_dr))
    out_layer = Dense(3, activation = 'softmax')(dr_steps)
    model = Model(inputs = [in_lay], outputs = [out_layer])  
    model.compile(
                optimizer=tf.optimizers.Adam(lr=0.0001),
                loss=categorical_focal_loss(gamma=2., alpha=.25),
                metrics=['categorical_accuracy',
                        tf.keras.metrics.Recall(),
                        tf.keras.metrics.Precision(),   
                        tf.keras.metrics.AUC(),
                        tfa.metrics.F1Score(num_classes=3, average="macro")
                       ])

In [ ]:
ATT_EF7= model.fit(
    train_dataset, 
    steps_per_epoch=train_labels.shape[0] // BATCH_SIZE,
    callbacks=[lr_callback],
    epochs=EPOCHS)

In [ ]:
import seaborn as sns
sns.set()
fig = plt.figure(0, (12, 4))

ax = plt.subplot(1, 2, 1)
sns.lineplot(ATT_EF7.epoch,ATT_EF7.history['categorical_accuracy'], label = 'train')
plt.title('Accuracy')
plt.tight_layout()

ax = plt.subplot(1, 2, 2)
sns.lineplot(ATT_EF7.epoch,ATT_EF7.history['loss'], label = 'train')
plt.title('Loss')
plt.tight_layout()
plt.show()

In [ ]:
model.evaluate(test_dataset)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
Y_pred = model.predict(test_dataset)
true_classes = valid.loc[:, ['N','C','G']].values
print('Confusion Matrix')
cm=confusion_matrix(true_classes.argmax(axis=1),Y_pred.argmax(axis=1))
cm

In [ ]:
plot_confusion_matrix(cm,classes)

In [ ]:
sensitivity, specificity= calculate_sensitivity_specificity(true_classes.argmax(axis=1),Y_pred.argmax(axis=1))
print ('Sensitivity:', sensitivity)
print ('Specificity:', specificity)

In [ ]:
with strategy.scope():
    in_lay = Input(shape=(CFG['inp_size'], CFG['inp_size'],3))
    base_model = tf.keras.applications.DenseNet201(weights='imagenet',
        input_shape=(CFG['inp_size'], CFG['inp_size'],3),
        include_top=False
                       )
    #base_model.load_weights("../input/efficientnet-keras-weights-b0b5/efficientnet-b5_imagenet_1000_notop.h5")
    pt_depth = base_model.get_output_shape_at(0)[-1]
    pt_features = base_model(in_lay)
    bn_features = BatchNormalization()(pt_features)
    
    # ici nous faisons un mécanisme d'attention pour activer et désactiver les pixels dans le GAP
    # lidee est baser sur cette explication 
    #1-http://akosiorek.github.io/ml/2017/10/14/visual-attention.html
    #2-https://machinelearningmastery.com/how-does-attention-work-in-encoder-decoder-recurrent-neural-networks/
    
    
    attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(Dropout(0.5)(bn_features))
    attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
    attn_layer = Conv2D(8, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
    attn_layer = Conv2D(1, 
                        kernel_size = (1,1), 
                        padding = 'valid', 
                        activation = 'sigmoid')(attn_layer)
    # diffusez sur toutes les chaînes
    # kernel_size  détermine les dimensions du noyau. Les dimensions courantes comprennent 1×1, 3×3, 5×5 et 7×7, qui peuvent être passées en (1, 1), (3, 3), (5, 5) ou (7, 7) tuples.
    # Il s'agit d'un nombre entier ou d'un tuple/liste de 2 nombres entiers, spécifiant la hauteur et la largeur de la fenêtre de convolution 2D.
    #  Ce paramètre doit être un nombre entier impair
    # pour plus de details sur cette partie (mask et use_bias ... ) il ya  une bonne explication sur geekforgeeks
    #https://www.geeksforgeeks.org/keras-conv2d-class/
    
    up_c2_w = np.ones((1, 1, 1, pt_depth))
    up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
                   activation = 'linear', use_bias = False, weights = [up_c2_w])
    up_c2.trainable = False
    attn_layer = up_c2(attn_layer)

    mask_features = multiply([attn_layer, bn_features])
    gap_features = GlobalAveragePooling2D()(mask_features)
    gap_mask = GlobalAveragePooling2D()(attn_layer)
    
    # pour tenir compte des valeurs manquantes du modèle d'attention
    # pour bien comprendre resaclegap il ya un bon exemple ici qui explique tellemnt bien cette partie 
    # https://codefellows.github.io/sea-python-401d5/lectures/rescaling_data.html
    
    gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
    gap_dr = Dropout(0.25)(gap)
    dr_steps = Dropout(0.25)(Dense(128, activation = 'relu')(gap_dr))
    out_layer = Dense(3, activation = 'softmax')(dr_steps)
    model_d201 = Model(inputs = [in_lay], outputs = [out_layer])  
    model_d201.compile(
                optimizer=tf.optimizers.Adam(lr=0.0001),
                loss=categorical_focal_loss(gamma=2., alpha=.25),
                metrics=['categorical_accuracy',
                        tf.keras.metrics.Recall(),
                        tf.keras.metrics.Precision(),   
                        tf.keras.metrics.AUC(),
                        tfa.metrics.F1Score(num_classes=3, average="macro")
                       ])

In [ ]:
d201= model_d201.fit(
    train_dataset, 
    steps_per_epoch=train_labels.shape[0] // BATCH_SIZE,
    callbacks=[lr_callback],
    epochs=EPOCHS)

In [ ]:
import seaborn as sns
sns.set()
fig = plt.figure(0, (12, 4))

ax = plt.subplot(1, 2, 1)
sns.lineplot(d201.epoch,d201.history['categorical_accuracy'], label = 'train')
plt.title('Accuracy')
plt.tight_layout()

ax = plt.subplot(1, 2, 2)
sns.lineplot(d201.epoch,d201.history['loss'], label = 'train')
plt.title('Loss')
plt.tight_layout()
plt.show()

In [ ]:
model_d201.evaluate(test_dataset)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
Y_pred = model_d201.predict(test_dataset)
true_classes = valid.loc[:, ['N','C','G']].values
print('Confusion Matrix')
cm=confusion_matrix(true_classes.argmax(axis=1),Y_pred.argmax(axis=1))
cm

In [ ]:
plot_confusion_matrix(cm,classes)

In [ ]:
sensitivity, specificity= calculate_sensitivity_specificity(true_classes.argmax(axis=1),Y_pred.argmax(axis=1))
print ('Sensitivity:', sensitivity)
print ('Specificity:', specificity)

# 5. Étude comparative

**l'évaluation des 4 modèles (vgg16,vgg19,DenseNet-201,EfficientNetB7) est basé sur Accuracy,Précision,Recall,F1_Score,AUC,Specificity,Sensitivity**

**Positifs vrais (TP)** - Ce sont les valeurs positives correctement prédites, ce qui signifie que la valeur de la classe réelle est oui et que la valeur de la classe prédite est également oui.

**Vrais négatifs (TN)** - Il s'agit des valeurs négatives correctement prédites, ce qui signifie que la valeur de la classe réelle est non et que la valeur de la classe prédite est également non.

**Faux positifs (FP) et faux négatifs (FN)** , ces valeurs se produisent lorsque votre classe réelle est en contradiction avec la classe prédite.

**Accuracy** - La précision est la mesure de performance la plus intuitive et il s'agit simplement d'un rapport entre l'observation correctement prédite et le total des observations.

**Accuracy  = TP+TN/TP+FP+FN+TN**

**Précision** - La précision est le rapport entre les observations positives correctement prédites et le total des observations positives prédites. 

**Précision = TP/TP+FP**

**Recall (sensibilité)**- Le Recall est le rapport entre les observations positives correctement prédites et toutes les observations de la classe réelle 

**Recall = TP/TP+FN**

**F1_Score** - Le score F1 est la moyenne pondérée de la précision et du rappel. Par conséquent, ce score tient compte à la fois des faux positifs et des faux négatifs. 

**F1_Score = 2*(Rappel * Précision) / (Rappel + Précision)**

**Sensitivity mesure la proportion de vrais positifs qui sont correctement identifiés**
**Specificity mesure la proportion de vrais négatifs qui sont correctement identifiés comme non**


| Methodes | Loss | Accuracy  | Precision | Recall  |F1_score  |specificity |sensitivity  |
| --- | --- | --- | --- | --- | --- | --- | --- |
| VGG16 |  0.9876 | 0.9884  | 0.9840|0.9874 |0.9991 |0.9991 |0.9991 |
| VGG19 | 0.9662  | 0.9807| 0.9484 |0.9657 |1.0000 |0.9991 |0.9991 |
|DenseNet201|0.9662  | 0.9807| 0.9484 |0.9657 |1.0000 |0.9991 |0.9991 |
|EfficientNetB7 |0.9662  | 0.9807| 0.9484 |0.9657 |1.0000 |0.9991 |0.9991 |